In [1]:
from tkinter import * 
#import tkinter as tk

In [2]:
import pygame
from pygame import mixer

pygame 2.0.1 (SDL 2.0.14, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

In [4]:
def playsong(self):#, b=None):
    #if b is None:
    #    b = self.a
    # Displaying Selected Song title
    self.track.set(self.playlist.get(ACTIVE))
    # Displaying Status
    self.status.set("-Playing")
    # Loading Selected Song
    pygame.mixer.music.load(self.playlist.get(ACTIVE))
    # Playing Selected Song
    pygame.mixer.music.play()
def stopsong(self):
    # Displaying Status
    self.status.set("-Stopped")
    # Stopped Song
    pygame.mixer.music.stop()
def pausesong(self):
    # Displaying Status
    self.status.set("-Paused")
    # Paused Song
    pygame.mixer.music.pause()
def unpausesong(self):
    # It will Display the  Status
    self.status.set("-Playing")
    # Playing back Song
    pygame.mixer.music.unpause()
def play_music():    
    #mixer.music.load("D:\Shreya\songs\Rise.mp3")
    mixer.music.play()
def pause_music():
    #mixer.music.load("D:\Shreya\songs\Rise.mp3")
    mixer.music.pause()

In [5]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance


#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

#function to identify the nearest neighbors
def nearestclass(neighbors):
    classVote = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

# define a function that will evaluate a model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

In [13]:
class MusicPlayer(object):
    def __init__(self,root):
    #def __init__(self):
        self.root = root
        # Title of the window
        self.root.title("MusicPlayer")
        # Window Geometry
        self.root.geometry("1000x200+200+200")
        # Initiating Pygame
        pygame.init()
        # Initiating Pygame Mixer
        pygame.mixer.init()
        # Declaring track Variable
        self.track = StringVar()
        # Declaring Status Variable
        self.status = StringVar()

        # Creating the Track Frames for Song label & status label
        trackframe = LabelFrame(self.root,text="Song Track",font=("times new roman",15,"bold"),bg="Navyblue",fg="white",bd=5,relief=GROOVE)
        trackframe.place(x=0,y=0,width=600,height=100)
        # Inserting Song Track Label
        songtrack = Label(trackframe,textvariable=self.track,width=20,font=("times new roman",24,"bold"),bg="Orange",fg="gold").grid(row=0,column=0,padx=10,pady=5)
        # Inserting Status Label
        trackstatus = Label(trackframe,textvariable=self.status,font=("times new roman",24,"bold"),bg="orange",fg="gold").grid(row=0,column=1,padx=10,pady=5)

        # Creating Button Frame
        buttonframe = LabelFrame(self.root,text="Control Panel",font=("times new roman",15,"bold"),bg="grey",fg="white",bd=5,relief=GROOVE)
        buttonframe.place(x=0,y=100,width=600,height=100)
        
        # Inserting Play Button
        playbtn = Button(buttonframe,text="PLAYSONG",command=play_music,width=10,height=1,font=("times new roman",16,"bold"),fg="navyblue",bg="pink").grid(row=0,column=0,padx=10,pady=5)
        
        # Inserting Pause Button
        pausebtn = Button(buttonframe,text="PAUSE",command=pause_music,width=8,height=1,font=("times new roman",16,"bold"),fg="navyblue",bg="pink").grid(row=0,column=1,padx=10,pady=5)
        # Inserting Unpause Button
        unpausebtn = Button(buttonframe,text="GENRE",command=GenreResult,width=10,height=1,font=("times new roman",16,"bold"),fg="navyblue",bg="pink").grid(row=0,column=2,padx=10,pady=5)
        # Inserting Stop Button
        stopbtn = Button(buttonframe,text="STOPSONG",command=lambda:stopsong,width=10,height=1,font=("times new roman",16,"bold"),fg="navyblue",bg="pink").grid(row=0,column=3,padx=10,pady=5)

        # Creating Playlist Frame
        songsframe = LabelFrame(self.root,text="Song Playlist",font=("times new roman",15,"bold"),bg="grey",fg="white",bd=5,relief=GROOVE)
        songsframe.place(x=600,y=0,width=400,height=200)
        # Inserting scrollbar
        scrol_y = Scrollbar(songsframe,orient=VERTICAL)
        # Inserting Playlist listbox
        self.playlist = Listbox(songsframe,yscrollcommand=scrol_y.set,selectbackground="gold",selectmode=SINGLE,font=("times new roman",12,"bold"),bg="silver",fg="navyblue",bd=5,relief=GROOVE)
        # Applying Scrollbar to listbox
        scrol_y.pack(side=RIGHT,fill=Y)
        scrol_y.config(command=self.playlist.yview)
        self.playlist.pack(fill=BOTH)
        # Changing Directory for fetching Songs
        os.chdir(r"C:\Users\swapj\Downloads\Data\genres_original\rock")
        #os.chdir(r"D:\Shreya\songs")
        # Fetching Songs
        songtracks = os.listdir()
        # Inserting Songs into Playlist
        for track in songtracks:

            self.playlist.insert(END,track)

        print(track)
        
        
        self.track.set(self.playlist.get(ACTIVE))
        print(self.playlist.get(ACTIVE))
        self.status.set("-Playing")
            # Loading Selected Song
        mixer.music.load(self.playlist.get(ACTIVE))
            
        #mixer.music.load("D:\Shreya/songs/" + track)
        mixer.music.pause()
        #playbtn = Button(buttonframe,text="PLAYSONG",command=play_music,width=10,height=1,font=("times new roman",16,"bold"),fg="navyblue",bg="pink").grid(row=0,column=0,padx=10,pady=5)
        
        

In [7]:
#import librosa, IPython
#import librosa.display
#file = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav'
#file = 'Downloads\Data\genres_original\disco\disco.00031.wav'
#signal, sr = librosa.load(file , sr = 22050) 
#IPython.display.Audio(signal, rate=sr)

#fileS= 'disco.00031.wav'
directory123 = 'C:/Users/swapj/Downloads/Data/genres_original'
#directory123 = 'D:/Shreya/songs'
fS = open("my1.dat", "wb")
iS = 0
folderS=" "
for folderS in os.listdir(directory123):
    #print(folderS)
    iS += 1
    if iS == 11:
        break
    for fileS in os.listdir(directory123+"/"+folderS):
        #print(fileS)
        try:
            (rate, sig) = wav.read(directory123+"/"+folderS+"/"+fileS)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            featureS = (mean_matrix, covariance, iS)
            pickle.dump(featureS, fS)
        except Exception as e: 
            print("Got an exception: ", e, 'in folder: ', folderS, ' filename: ', fileS)
fS.close()

Got an exception:  File format b''... not understood. in folder:  disco  filename:  my.dat
Got an exception:  File format b'\x80\x03cn'... not understood. in folder:  disco  filename:  my1.dat
Got an exception:  File format b'\xcb\x15\x1e\x16'... not understood. in folder:  jazz  filename:  jazz.00054.wav


In [8]:
#split dataset into train and test set
dataset = []

def loadDataset(filename, split, trset, teset):
    with open('my1.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset('my1.dat', 0.66, trainingSet, testSet)



# Make the prediction using KNN(K nearest Neighbors)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 20)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.6501547987616099


In [14]:
from collections import defaultdict
results = defaultdict(int)

directory12 = 'C:/Users/swapj/Downloads/Data/genres_original/rock'

i = 1
for folder in os.listdir(directory12):
    results[i] = folder
    #print(directory)
    #print(folder)
    #print(results[i])
    i += 1

In [15]:
pred = nearestclass (getNeighbors (dataset, featureS, 5))
#print(results[pred])

partitioned_string = results[pred].partition('.')
#print(partitioned_string)
before_first_period = partitioned_string[0]

print(before_first_period)
def GenreResult(): 
    import ctypes  # An included library with Python install.   
    ctypes.windll.user32.MessageBoxW(0, (before_first_period), "Genre", 1)

rock


In [16]:
root = Tk() # In order to create an empty window
# Passing Root to MusicPlayer Class
MusicPlayer(root)


root.mainloop()

rock.00099.wav
rock.00000.wav
